In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"
data = pd.read_csv('../covtype.data', header=None)

columns = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]
wilderness_areas = [f'Wilderness_Area_{i}' for i in range(1, 5)]
soil_types = [f'Soil_Type_{i}' for i in range(1, 41)]
columns.extend(wilderness_areas)
columns.extend(soil_types)
columns.append('Cover_Type')
data.columns = columns
data

In [ ]:
cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points', 'Cover_Type']

In [ ]:
for i, column in enumerate(cols):
    fig, ax = plt.subplots()
    sns.boxplot(data=data[column], ax=ax).set(title=f"{column.upper()} boxplot", xlabel=f"{column}", ylabel=f"Value of {column}")
    ax.set_title(column)
    sns.despine()
    plt.show()

In [ ]:
# Calculate the z-scores of each column - measure that represents the number of standard deviations a data point is from the mean of the dataset
z_scores = (data - data.mean()) / data.std()
# Set the threshold for the z-score
threshold = 3
# Remove any rows where the z-score is greater than the threshold
data = data[(np.abs(z_scores) < threshold).all(axis=1)]

# Interquartile Range (IQR)
# Q1 = data.quantile(0.25)
# Q3 = data.quantile(0.75)
# IQR = Q3 - Q1
# data = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]


In [ ]:
print(len(data))

In [ ]:
data_subset = data[cols]
correlation_matrix = np.corrcoef(data_subset.values.T)

fig, ax = plt.subplots(figsize=(7, 7))
sns.set(font_scale=1.1)
sns.heatmap(data=correlation_matrix, square=True, cbar=True, annot=True, annot_kws={'size': 10}, xticklabels=cols, yticklabels=cols, fmt=".2f", linewidth=.5, cmap=sns.cubehelix_palette(as_cmap=True))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("Cover_Type", axis=1), data["Cover_Type"], test_size=0.2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(7, activation="softmax")
])

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)